In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice

import ast

import sentencepiece as sp

from sklearn.metrics.pairwise import cosine_similarity
import arabicstopwords.arabicstopwords as stp

from arabert.preprocess import ArabertPreprocessor

import tkseem as tk
import gensim

In [2]:
bert_embedder = gensim.models.Word2Vec.load("models/bert_word2vec_model.model")

model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name)

[2024-06-05 19:53:05,562 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [11]:
df = pd.read_csv('all_poems_bert_processed.csv').sample(100_000, random_state=42).reset_index()
df.head()

,index,poem_text,bert_t
0,492926,بما بعينيك من فتون ومن فتور بها وسحر,"['ما', 'عيني', 'من', 'تون', 'من', 'فتور', 'سحر']"
1,1826254,إن خطو الخطوب تقصر عمن ظل نعماك فوقه ممدود,"['إن', 'خطو', 'خطوب', 'تقصر', 'عنمن', 'ظل', 'ن..."
2,430787,وغار منا حسدا يريد أنا نفترق,"['غار', 'من', 'حسد', 'يريد', 'أنا', 'نفترق']"
3,1796353,وأسخط لاستمرار هجرك ساعة وتغلب أشواقي فأرجع...,"['أسخط', 'استمرار', 'هجر', 'ساع', 'تغلب', 'أشو..."
4,1711449,فأفاد الفواد علما خفيا ليس تحصيه في المقال ...,"['أفاد', 'فواد', 'علم', 'خفي', 'ليس', 'تحصي', ..."


In [12]:
df['bert_t'] = df['bert_t'].apply(ast.literal_eval)
df['bert_t']

0                       [ما, عيني, من, تون, من, فتور, سحر]
1        [إن, خطو, خطوب, تقصر, عنمن, ظل, نعما, فوق, ممدود]
2                         [غار, من, حسد, يريد, أنا, نفترق]
3        [أسخط, استمرار, هجر, ساع, تغلب, أشواق, أرجع, ر...
4        [أفاد, فواد, علم, خفي, ليس, تحصي, في, مقال, رقوم]
                               ...                        
99995                        [كان, ورد, جني, في, جوف, ماء]
99996    [خول, أطلال, برق, ثهمد, صرف, إلى, مدح, نبي, محمد]
99997    [أسمر, عسال, إذا, احتدم, وغى, تصدق, من, زاد, أ...
99998            [يا, نديم, طيب, نفس, فيوم, أحسن, من, أمس]
99999    [عج, على, رسم, ربع, تجر, من, صيف, جشاء, حنين, ...
Name: bert_t, Length: 100000, dtype: object

In [57]:
embedding_size = bert_embedder.wv.__getitem__('دجاج').shape[0]

def bert_embed(tokens_poem):
    weights = [0.3 if stp.is_stop(token) else 1 for token in tokens_poem if token in bert_embedder.wv.key_to_index]
    embedded_tokens = np.array([bert_embedder.wv.__getitem__(token) for token in tokens_poem if token in bert_embedder.wv.key_to_index])
    
    if(len(embedded_tokens) < 1):
        return np.zeros(embedding_size)
    
    if sum(weights) > 0:
        return np.average(embedded_tokens, axis=0, weights=weights)
    return np.mean(embedded_tokens, axis=0)

# TODO: Try hnaya tdire weighted embedding 3la 7sab l idf
# Note: Kayane TfidVectorizer f sklearn

# TODO: Sinon nsayo idea wa7do5ra:
#         Basically, mandirouch kml document embedding
#         Instead, njibo f kol document lmost similar words m3a l words nta3 l query nta3na
#         Wndiro weighted average 3la lsimilarities

df['bert_embd'] = df['bert_t'].apply(bert_embed)
df['bert_embd']

0        [0.230827799177773, 0.18682234166633518, 0.935...
1        [-0.3161160061436315, 0.6250512405118396, -0.1...
2        [-0.39759015956002736, 0.23858210995145465, -0...
3        [-0.4520987756550312, 0.6223803540342487, -0.5...
4        [-0.09296007611249622, 0.5819293840935356, 0.6...
                               ...                        
99995    [0.3202946474370749, 0.7080051914343368, 0.666...
99996    [0.14820309432156115, 0.7618179784481783, 0.80...
99997    [-0.3465792702207732, 0.5089626119710332, 0.08...
99998    [-0.01343505147655132, 0.3742859497413797, 0.0...
99999    [0.8325582419543766, 0.7073989711095428, 0.550...
Name: bert_embd, Length: 100000, dtype: object

In [58]:
import faiss

arrays = df['bert_embd'].values.tolist()
embeddings = np.stack(arrays, axis=0)

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_size)
index.add(embeddings)
print(f'Total vectors in the index: {index.ntotal}')

Total vectors in the index: 100000


In [59]:
query = 'بق بق نحن نغرق'
query_vector = bert_embed(query).reshape(1, -1)
distances, indices = index.search(query_vector, 20)
df.loc[indices.flatten()]

,index,poem_text,bert_t,bert_embd
84658,128258,زبالة من دونه والشقو ق والثعلبية والأجفر,"[زبال, من, دون, شقو, ق, ثعلبي, أجفر]","[-0.026196868900133643, 0.3030838248681049, -0..."
10327,29409,نغلي السباء بكل عا تقة شمول ما صحونا,"[نغل, سباء, كل, عا, تق, شمول, ما, صحون]","[-0.3285141520512601, -0.10470241765406998, 0...."
21660,1829676,وصفوت من كل الجوانب نية,"[صفوت, من, كل, جوانب, ني]","[-0.4876017469084925, 0.3946862218694554, -0.1..."
88511,670192,وطفقت أدكر العرا ق فعاد صفوي ذا كدور,"[طفق, أدكر, عرا, ق, عاد, صفوي, ذا, دور]","[-0.01486827418781243, 0.49425849064181626, -0..."
12121,989567,مخرتك القرون تشمر عن سا ق بعيد الخطى قوي ال...,"[مخر, قرون, تشمر, عن, سا, ق, بعيد, خطى, قوي, س...","[0.21109143956456072, 0.21446026982956154, -0...."
64598,809670,أيها المدلج الذي فلق البح ر انبعاثا وزايل ا...,"[أيها, مدلج, الذي, فلق, بح, ر, انبعاث, زايل, أ...","[-0.2922515758013321, 0.23159735207840543, 0.2..."
63088,254873,كمن دحرج در البح ر في بخر البلاليع,"[من, دحرج, در, بح, ر, في, بخر, بلاليع]","[0.2730579573219105, 0.4382209691329527, -0.00..."
78922,441452,إذا ما سقسق العصفو ر في أعشاشه الغن,"[إذا, ما, سقسق, عصفو, ر, في, أعشاش, غن]","[0.13243882209506064, 0.47015729528807454, -0...."
29411,626046,هو إما الذعاف رقرقه الص ل لحاويه أو هناء النقب,"[هو, إما, ذعاف, رقرق, الص, حاوي, أو, هناء, نقب]","[-0.03159702013152233, 0.6936065750061602, 0.6..."
90249,216457,بدلا ما استبدل الدا ئر فيها والمدير,"[بدل, ما, استبدل, دا, ئر, في, مدير]","[-0.28467065253893714, 0.49260863475501543, -0..."


In [61]:
query = 'يا الهي'
query_vector = bert_embed(query).reshape(1, -1)
distances, indices = index.search(query_vector, 20)
df.loc[indices.flatten()]

,index,poem_text,bert_t,bert_embd
66580,1400823,إغنوسة ميليتس ولكتس ليكستس ريتية وفستس,"[إغنوسة, ميليتس, كتس, ليكستس, ريتي, فستس]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
90555,1782999,والساديه,[ساديه],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
85853,931926,واقتلوني,[واقتلوني],"[-0.03163081407546997, -0.0756298303604126, 0...."
761,263167,فبأنوارك نضحى وبسقياك نغام,"[فبأنوارك, نضحى, وبسقياك, نغام]","[-0.025904315523803234, 0.0223323394311592, 0...."
2894,307379,يهمدن للاجراس والتشوير,"[يهمدن, اجراس, تشوير]","[0.02455404121428728, -0.04952765675261617, 0...."
68600,1785793,هم عتاه,[عتاه],"[0.023519137874245644, -0.0029414095915853977,..."
48600,410619,يؤاتيني فأرجوه ويجفوني فأخشاه,"[يؤاتيني, فأرجوه, ويجفوني, فأخشاه]","[-0.12996279634535313, 0.013652224035467952, -..."
12475,1268862,ودعنني وتركنني كالمومياء الفانيه,"[دعنني, وتركنني, مومياء, الفاني]","[-0.07891709264367819, 0.01924478355795145, 0...."
15672,1210781,متقنعين معافرا متقمصين الماديه,"[متقنعين, معافر, متقمصين, ماديه]","[0.09398094564676285, -0.04538431018590927, 0...."
69827,1185561,جويريات دميعي خويدمات ستيته,"[جويري, دميعي, خويدمات, ستيت]","[-0.10075093805789948, 0.12310848385095596, 0...."


### OLD CODE

In [27]:
query = 'خشيت موج البحر'

indices = search_bert(query, N=4)
for i, index in enumerate(indices):
    print((i+1), ': ', df.at[index, 'poem_text'])

1 :  ما زال ينهلني الأجاج مكررا    ويعل غيري بالزلال الرائق
2 :  إذ نحن حيان حل الناس بينهما    وقد جهدنا لهم بالجمع واجتهدوا
3 :  أيا قد سما فهما    فلا قبلا ولا عقبا
4 :  لقد قدرت سرداوضوعف نسجها    واحرزها داود من بعده ذخرا


In [9]:
query = 'الحرية ثمنها غالي'

verses = search_bert(query).values.tolist()
for i, verse in enumerate(verses):
    print((i+1), ': ', verse)

1 :  وتشتري نفسه المعروف بالثمن ال    غالي ولو أنها كانت من الثمن
2 :  واحمل بقلبك حبها    واجعل هواك لها ابتهالا
3 :  ولصك المدعي قد وقعا    


In [30]:
query = 'الحرية ثمنها غالي'

indices = search_bert(query, N=7)
for i, index in enumerate(indices):
    print((i+1), ': ', df.at[index, 'poem_text'])

1 :  وإن قال البغيض بلا احتشام    فلان منك أعرف بالمقال
2 :  مولاي أجر حليف سقم    قد جد به وأنت تلعب
3 :  يعطي البشير    
4 :  أغالي بها في سومها فتردني    إلى ثمن بخس زهيد مطفف
5 :  شأنها في التأريخ اكبر شأن    حبذا في التأريخ تلك الشؤون
6 :  إذا روهنوا في حلبة أحرزوا المدى    وأحيوا لنا ذكر الوجيه ولاحق
7 :  يا أيها الملك الذي آمالنا    ما برحت في جوده تصدق


In [10]:
query = 'عنترة'

verses = search_bert(query).values.tolist()
for i, verse in enumerate(verses):
    print((i+1), ': ', verse)

1 :  فإن جاء ذكر الميتين صبابة    على شعبهم لا شك أنك ذاكرى
2 :  أرى في النوم رمحا أو سنانا    فأسلج في الفراش على مكاني
3 :  بدر الهدى قطر الندى بحر الجدى    جالي الصدا مجلي العدى الفجار


In [ ]:
query = 'عنترة'

verses = search_avg_sim(query).values.tolist()
for i, verse in enumerate(verses):
    print((i+1), ': ', verse)

In [29]:
query = 'ليت شبابي يرجع'

indices = search_bert(query, N=4)
for i, index in enumerate(indices):
    print((i+1), ': ', df.at[index, 'poem_text'])

1 :  أقصر فلا ذاك الخليل بآيب    يوما ولا ذاك الشباب يعاد
2 :  وما دهري اطبأنك غير أني    بنى لي والدي بيتا يفاعا
3 :  قسوة فيهم وفرط جفا    لم يخف مرميهم رامي
4 :  أم عليه تغار من عين شمس    مد من دونها عليه الغطاء


In [ ]:
query = 'ليت شبابي يرجع'

verses = search_avg_sim(query).values.tolist()
for i, verse in enumerate(verses):
    print((i+1), ': ', verse)